In [ ]:
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

from IPython.display import clear_output, display
from IPython.display import Image as IPyImage

IMAGE_TOPIC = "/yolo/annotated"   # 필요하면 /camera/color/image_raw 로 변경 가능

# ROS 노드 초기화 (이미 init 되어 있으면 그냥 넘어감)
if not rospy.core.is_initialized():
    rospy.init_node("jupyter_yolo_viewer_sync", anonymous=True)

bridge = CvBridge()
print("Ready. Topic =", IMAGE_TOPIC)


In [ ]:
import rospy

try:
    while not rospy.is_shutdown():
        # 토픽에서 한 프레임 받아오기 (동기식)
        msg = rospy.wait_for_message(IMAGE_TOPIC, Image, timeout=1.0)
        print("got msg", msg.header.stamp.to_sec())


        # ROS Image -> OpenCV BGR
        frame_bgr = bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8")

        # JPEG 인코딩
        ok, buf = cv2.imencode(".jpg", frame_bgr)
        if not ok:
            continue

        # 노트북 셀 출력 갱신
        clear_output(wait=True)
        display(IPyImage(data=buf.tobytes(), format='jpeg'))

except rospy.ROSException as e:
    print("ROS wait_for_message timeout/error:", e)
except KeyboardInterrupt:
    print("Stopped by user")
